1.05 Change log: 
* tbl_dct change to token-[queries] pairs for a more robust development
* optimizes the query logic, now only evaluates the ones whose previous gate returns True
* simplifies menu option, now describes each possible redirect link with the name in format of {name of the distribution}-{label of the application}

1.04 Change log: 
<br>
...
<br>
* AVENGERS!!!!! 
<br>
* **ASSEMBLE**

1.03 Change log:
<br>
* add label and description to the menu

1.02 Change log:
<br>
* mapping functionality complete. Now conherently maps from json to token to desired url based on users choice of tokens and specific transfer urls.

1.01 Change log:
<br>
* Changed the return type of features to tokens from a set of tokens to a list of dictionaries of token-url pairs. Urls in one dict should be the same.
<br>
* Changed the token aggregation logic in dists_to_tokens(). The result is no longer a union of different set of tokens (due to above change) but a concatnation of the dicts

# Data Discovery Studio Jupyter Notebook Dispatcher

Welcome to the Data Discovery Studio's Jupyter Notebook Server.

This environment allows you to process data referenced from the DDS discovery interface. In a typical case, DDS document ID will be passed to the DDS notebook dispatcher, and then an appropriate notebook (for this data type) will be called to retrieve, analyze, visualize and otherwise process the data referenced in the DDS metadata record.

Example call with Document ID from DDS: CinergiDispatch.ipynb?documentId=61cc7f6afb5246d2be41811e94a1a8ea

In addition, if you have a collection of Jupyter notebooks organized in a similar github repository (e.g., to retrieve and visualize  additional types of dat,a or to offer additional processing), you can clone this repository into a running container and execute your copy of the dispatcher for the same document ID (see cell 2a.) 

## 1. Execute the cells below to get parameters from DDS and select a notebook for processing ###



In [1]:
!pip install lxml

In [2]:
from __future__ import print_function
import ntpath
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import webbrowser
from lxml import etree  #supposed to be better than xml.etree
import json
import pandas as pd
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import requests

In [3]:
%%javascript
function getQueryStringValue (key)
{  
    return unescape(window.location.search.replace(new RegExp("^(?:.*[&\\?]" + escape(key).replace(/[\.\+\*]/g, "\\$&") + "(?:\\=([^&]*))?)?.*$", "i"), "$1"));
}
IPython.notebook.kernel.execute("documentID='".concat(getQueryStringValue("documentId")).concat("'"));
IPython.notebook.kernel.execute("user='".concat(getQueryStringValue("user")).concat("'"));
IPython.notebook.kernel.execute("full_notebook_url='" + window.location + "'"); 

<IPython.core.display.Javascript object>

In [4]:
# Only run this cell when it is for demo!!!!!!!
sample_id = 'e3619c5df2644204b67f51f48525a0b1'
documentID = sample_id

## 2. Print out parameters passed to Jupyter: ##

In [5]:
# Check if the parameters are correct
from __future__ import print_function
import ntpath
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import webbrowser
url_partitioned = full_notebook_url.partition('/CinergiDispatch')
base_url = url_partitioned[0];

print("User: ",user)
print("DocumentID: ", documentID)
print("Full notebook URL: ", full_notebook_url)

User:  
DocumentID:  e3619c5df2644204b67f51f48525a0b1
Full notebook URL:  https://datahub.ucsd.edu/user/shl636/notebooks/Cinergi_testings/token_assignment_v1.05.ipynb


### 2.1 Parsing XML metadata

In [6]:
NSMAP = {"gmi":"http://www.isotc211.org/2005/gmi" ,
    "gco":"http://www.isotc211.org/2005/gco" ,
    "gmd":"http://www.isotc211.org/2005/gmd" ,
    "gml":"http://www.opengis.net/gml" ,
    "gmx":"http://www.isotc211.org/2005/gmx" ,
    "gts":"http://www.isotc211.org/2005/gts" ,
    "srv":"http://www.isotc211.org/2005/srv" ,
    "xlink":"http://www.w3.org/1999/xlink"}

In [7]:
#construct url for metadata
catalogURL = "http://datadiscoverystudio.org/geoportal/"
metadataURLx=catalogURL + 'rest/metadata/item/' + documentID + '/xml'
#metadata = requests.get(metadataURLx)

print("metadata URL: ", metadataURLx)

metadata URL:  http://datadiscoverystudio.org/geoportal/rest/metadata/item/e3619c5df2644204b67f51f48525a0b1/xml


In [8]:
#tree is an element tree
tree = etree.parse(metadataURLx)
#root = etree.tostring(tree.getroot())
root = tree.getroot()
docinfo = tree.docinfo
print(docinfo.xml_version)

1.0


In [9]:
title=tree.find("//gmd:identificationInfo//gmd:title/gco:CharacterString",namespaces=NSMAP).text
print(title)

Colorado Active Faults


In [10]:
# define distribution object with CI_ONlineResource elements, plus the distributor organization 
#  and list of possible formats

class DistObj:
    def __init__(self,aname):
        self.name = aname
        self.url = ''
        self.description = ''
        self.protocol = ''
        self.appprofile = ''
        self.functioncode = ''
        self.functiontext = ''
        self.distorg = ''
        self.formatlist = []
        
    def dump(self):
        return {"adistobj": {'name': self.name,
                               'url': self.url,
                               'description': self.description,
                               'protocol': self.protocol,
                                'appprofile': self.appprofile,
                               'functioncode': self.functioncode,
                               'functiontext': self.functiontext,
                               'distorg': self.distorg,
                               'formatlist' : self.formatlist                            
                             }}

In [11]:
# # development of logic for reading the xml tree
# for  dist in tree.getiterator("{http://www.isotc211.org/2005/gmd}MD_Distribution"):
#     if dist.find("gmd:distributionFormat/gmd:MD_Format/gmd:name/gco:CharacterString",namespaces=NSMAP) is not None: 
#         # this code is not handling multiple formats under a distribution at this point. 
#         # have to consider how to deal with binding between formats and digital transfer options
        
#         dist_format = dist.find("gmd:distributionFormat/gmd:MD_Format/gmd:name/gco:CharacterString",namespaces=NSMAP).text
#     else:
#         dist_format = ''
#     print(dist_format)
#     if dist.find("gmd:distributor/gmd:MD_Distributor//gmd:organisationName/.",namespaces=NSMAP) is not None:
#         distorg =dist.xpath("gmd:distributor//gmd:organisationName/child::node()/text()",namespaces=NSMAP)[0]
#         print('distribution org: ' + distorg)
#     else:
#         distorg = ''
        
#     if dist.find("gmd:distributor/gmd:MD_Distributor//gmd:MD_DigitalTransferOptions//gmd:CI_OnlineResource.",namespaces=NSMAP) is not None:
#         distonline =dist.find("gmd:distributor/gmd:MD_Distributor//gmd:MD_DigitalTransferOptions//gmd:CI_OnlineResource",namespaces=NSMAP)
#     else:
#         distonline = ''
        
#     print(distonline)

In [12]:
distlist=[]  # this will be a list of distribution objects

# plan -- iterate over the CI_OnlineResource Elements that are in MD_DigitalTransfer Options
for  elt in tree.getiterator("{http://www.isotc211.org/2005/gmd}MD_DigitalTransferOptions"):
    # only want OnlineResources that are in distribution//MD_DigitalTransferOptions
    #  TBD-- figure out what to do with CI_OnlineResource inside SV_OperationMetadata
    
#iterate through CI_OnlineResource elements
    for onlineres in elt.getiterator("{http://www.isotc211.org/2005/gmd}CI_OnlineResource"):
        
        if ((onlineres.find("gmd:linkage/gmd:URL",namespaces=NSMAP) is not None) and
            (onlineres.find("gmd:linkage/gmd:URL",namespaces=NSMAP).text is not None)  ):
            theURL=onlineres.find("gmd:linkage/gmd:URL",namespaces=NSMAP).text
        else:
            theURL = 'empty'
        
        #print('theURL ' + theURL + '\n')
        
        if (onlineres.find("gmd:name/gco:CharacterString",namespaces=NSMAP) is not None):
            thename=onlineres.find("gmd:name/gco:CharacterString",namespaces=NSMAP).text
        else:
            thename=''
        
        if (onlineres.find("gmd:description/gco:CharacterString",namespaces=NSMAP) is not None):
            thedescription=onlineres.find("gmd:description/gco:CharacterString",namespaces=NSMAP).text
        else:
            thedescription=''
            
        if (onlineres.find("gmd:protocol/gco:CharacterString",namespaces=NSMAP) is not None):
            theprotocol=onlineres.find("gmd:protocol/gco:CharacterString",namespaces=NSMAP).text
        else:
            theprotocol=''
        
        if (onlineres.find("gmd:applicationProfile/gco:CharacterString",namespaces=NSMAP) is not None):
            theappprofile=onlineres.find("gmd:applicationProfile/gco:CharacterString",namespaces=NSMAP).text
        else:
            theappprofile=''
            
        if (onlineres.find("gmd:function/gmd:CI_OnLineFunctionCode",namespaces=NSMAP) is not None):
            thefunctioncode=onlineres.find("gmd:function/gmd:CI_OnLineFunctionCode",namespaces=NSMAP).get("codeListValue")
        else:
            thefunctioncode=''
            
        if (onlineres.find("gmd:function/gmd:CI_OnLineFunctionCode",namespaces=NSMAP) is not None):    
            thefunctiontext=onlineres.find("gmd:function/gmd:CI_OnLineFunctionCode",namespaces=NSMAP).text
        else:
            thefunctiontext=''
            
        #print('\n Distribution: name-%s;\n  url- %s; \n  description--%s; \n   protocol-%s, app profile- %s; function- %s; %s' %
        #      (thename,theURL,thedescription,theprotocol,theappprofile,thefunctioncode,thefunctiontext))

        #Handle format and distributor organization
# have to figure out who is the distributor
#check to see if have multiple distributors, if so they should have distributor formats and transfer options
#   if they don't then assume all formats apply to all distributions
        formatlist = []  #initialize

        if len(onlineres.xpath("./ancestor::gmd:MD_Distribution/gmd:distributor",namespaces=NSMAP)) <= 1:
     #have zero or one distributor; 
            distorg = ''
            if len(onlineres.xpath("./ancestor::gmd:MD_Distribution/gmd:distributor",namespaces=NSMAP)) == 1:
                dist = onlineres.xpath("./ancestor::gmd:MD_Distribution/gmd:distributor",namespaces=NSMAP)[0]
                if len(dist.xpath("gmd:MD_Distributor//gmd:organisationName",namespaces=NSMAP)) > 0:
                    distorg =dist.xpath("gmd:MD_Distributor//gmd:organisationName/child::node()/text()",namespaces=NSMAP)[0]
            #print("distorg: " + distorg)                        
    #get formats. Formats might be on Distribution, Distributor, or DigitalTransferOption
            thedistformats = onlineres.xpath("./ancestor::gmd:MD_Distribution/gmd:distributionFormat",namespaces=NSMAP)
            #get formats at gmd:MD_Distribution/gmd:distributionFormat/gmd:MD_Format       
            for aformat in thedistformats:
                if ((aformat.find("gmd:MD_Format/gmd:name/gco:CharacterString",namespaces=NSMAP) is not None) and
                    (aformat.find("gmd:MD_Format/gmd:name/gco:CharacterString",namespaces=NSMAP).text not in formatlist)):
                    formatlist.append(aformat.find("gmd:MD_Format/gmd:name/gco:CharacterString",namespaces=NSMAP).text)

            thedistformats = onlineres.xpath("./ancestor::gmd:MD_DigitalTransferOptions/gmd:distributionFormat",namespaces=NSMAP)
            #get formats on the parent gmd:MD_DigitalTransferOptions       
            for aformat in thedistformats:
                if (( aformat.find("gmd:MD_Format/gmd:name/gco:CharacterString",namespaces=NSMAP) is not None) and
                            (aformat.find("gmd:MD_Format/gmd:name/gco:CharacterString",namespaces=NSMAP).text not in formatlist)):
                    formatlist.append(aformat.find("gmd:MD_Format/gmd:name/gco:CharacterString",namespaces=NSMAP).text)


            thedistformats = onlineres.xpath("./ancestor::gmd:MD_Distribution//gmd:distributorFormat",namespaces=NSMAP)
            #get formats on the gmd:MD_Distributor; the transfer options might not be child of distributor    
            for aformat in thedistformats:
                if ((aformat.find("gmd:MD_Format/gmd:name/gco:CharacterString",namespaces=NSMAP) is not None) and
                       (aformat.find("gmd:MD_Format/gmd:name/gco:CharacterString",namespaces=NSMAP).text in formatlist)):
                    formatlist.append(aformat.find("gmd:MD_Format/gmd:name/gco:CharacterString",namespaces=NSMAP).text)
  
        elif len(onlineres.xpath("./ancestor::gmd:MD_Distributor",namespaces=NSMAP)) == 1:
            #MD_DigitalTransferOptions is child of MD_Distributor; there are multiple distributors
            distorg = ''
            dist = onlineres.xpath("./ancestor::gmd:MD_Distributor",namespaces=NSMAP)[0]
            if len(dist.xpath("gmd:distributorContact//gmd:organisationName",namespaces=NSMAP)) > 0:
                distorg =dist.xpath("gmd:distributorContact//gmd:organisationName/child::node()/text()",namespaces=NSMAP)[0]
            #print("distorg: " + distorg) 
            
            
            #check if they have distributorFormat 
            thedistformats = onlineres.xpath("./ancestor::gmd:MD_Distributor/gmd:distributorFormat",namespaces=NSMAP)
            #get formats on the gmd:MD_Distributor; note in this case look for specific distributor that is parent 
            #   of the digital transfer options/online resource.
            for aformat in thedistformats:
                if ((aformat.find("gmd:MD_Format/gmd:name/gco:CharacterString",namespaces=NSMAP) is not None) and
                       (aformat.find("gmd:MD_Format/gmd:name/gco:CharacterString",namespaces=NSMAP).text in formatlist)):
                    formatlist.append(aformat.find("gmd:MD_Format/gmd:name/gco:CharacterString",namespaces=NSMAP).text)
                    
                    
            thedistformats = onlineres.xpath("./ancestor::gmd:MD_DigitalTransferOptions/gmd:distributionFormat",namespaces=NSMAP)
            #get formats specific to the parent gmd:MD_DigitalTransferOptions       
            for aformat in thedistformats:
                if (( aformat.find("gmd:MD_Format/gmd:name/gco:CharacterString",namespaces=NSMAP) is not None) and
                            (aformat.find("gmd:MD_Format/gmd:name/gco:CharacterString",namespaces=NSMAP).text not in formatlist)):
                    formatlist.append(aformat.find("gmd:MD_Format/gmd:name/gco:CharacterString",namespaces=NSMAP).text)
            
            thedistformats = onlineres.xpath("./ancestor::gmd:MD_Distribution/gmd:distributionFormat",namespaces=NSMAP)
            #get formats at gmd:MD_Distribution/gmd:distributionFormat; assume these apply to all digital transfer options       
            for aformat in thedistformats:
                if ((aformat.find("gmd:MD_Format/gmd:name/gco:CharacterString",namespaces=NSMAP) is not None) and
                    (aformat.find("gmd:MD_Format/gmd:name/gco:CharacterString",namespaces=NSMAP).text not in formatlist)):
                    formatlist.append(aformat.find("gmd:MD_Format/gmd:name/gco:CharacterString",namespaces=NSMAP).text)
                    
        else:
            # multiple distributors, but digital transfer options are not associated with specific distributor
            
            distorg = ''
            # arbitrarily take the first distributor organization
            dist = onlineres.xpath("./ancestor::gmd:MD_Distribution/gmd:distributor",namespaces=NSMAP)[0]
            if len(dist.xpath("gmd:MD_Distributor//gmd:organisationName",namespaces=NSMAP)) > 0:
                distorg =dist.xpath("gmd:MD_Distributor//gmd:organisationName/child::node()/text()",namespaces=NSMAP)[0]
            #print("distorg: " + distorg) 
            
            #  assume all distributors offer all digital transfer options and formats that are child of distribution
            thedistformats = onlineres.xpath("./ancestor::gmd:MD_DigitalTransferOptions/gmd:distributionFormat",namespaces=NSMAP)
            #get formats specific to the parent gmd:MD_DigitalTransferOptions       
            for aformat in thedistformats:
                if (( aformat.find("gmd:MD_Format/gmd:name/gco:CharacterString",namespaces=NSMAP) is not None) and
                            (aformat.find("gmd:MD_Format/gmd:name/gco:CharacterString",namespaces=NSMAP).text not in formatlist)):
                    formatlist.append(aformat.find("gmd:MD_Format/gmd:name/gco:CharacterString",namespaces=NSMAP).text)
            
            thedistformats = onlineres.xpath("./ancestor::gmd:MD_Distribution/gmd:distributionFormat",namespaces=NSMAP)
            #get formats at gmd:MD_Distribution/gmd:distributionFormat; assume these apply to all digital transfer options       
            for aformat in thedistformats:
                if ((aformat.find("gmd:MD_Format/gmd:name/gco:CharacterString",namespaces=NSMAP) is not None) and
                    (aformat.find("gmd:MD_Format/gmd:name/gco:CharacterString",namespaces=NSMAP).text not in formatlist)):
                    formatlist.append(aformat.find("gmd:MD_Format/gmd:name/gco:CharacterString",namespaces=NSMAP).text)
        
        thisdistobj = DistObj(thename)
        #thisdistobj.name = thename
        #print("theName: " + thename)
        thisdistobj.url = theURL
        thisdistobj.description = thedescription
        thisdistobj.protocol = theprotocol
        thisdistobj.appprofile = theappprofile
        thisdistobj.functioncode = thefunctioncode
        thisdistobj.functiontext = thefunctiontext
        thisdistobj.distorg = distorg
        thisdistobj.formatlist = formatlist
                
        distlist.append(thisdistobj)                    

    
    
json_dump_str = json.dumps([theobj.dump() for theobj in distlist])

In [13]:
import pandas as pd
import numpy as np
import re
import json
from ipywidgets import interact, interactive
import ipywidgets as widgets

### Setting up Table 1 and 2

Building Table 1 (meta data to tokens)

In [14]:
#dissect Steve's logic into different levels of queries
queries_lvl1 = []
#wfs
queries_lvl1 += ["theprotocol.lower().find('wfs')>-1 or theURL.lower().find('service=wfs')>-1"]
#wms
queries_lvl1 += ["theprotocol.lower().find('wms')>-1 or (theURL.lower().find('service=wms')>-1 and theURL.lower().find('request=kml')==-1)"]
#kmlclient
queries_lvl1 += ["thedescription.lower().find('kml download')>-1 or (theURL.lower().find('request=kml')>-1 and theURL.lower().find('mode=download')>-1) or theURL.lower().find('.kml')>-1 or theURL.lower().find('.kmz')>-1"]
#magic
queries_lvl1 += ["(theappprofile.lower().find('pymag')>-1) or (theappprofile.lower().find('magic:tsv:multipart')>-1) or (thename.lower().find('magic multipart tab separated values')>-1 )"]
#webbrowser
queries_lvl1 += ["(theURL.lower().find('.html')>-1 ) or (theURL.lower().find('.pdf')>-1) or (theURL.lower().find('.htm')>-1 ) or (theprotocol.lower().find('http')>-1)"]
#nwis_rdb
queries_lvl1 += ["(theURL.lower().find('/nwis/qwdata')>-1) or (theURL.lower().find('/nwis/gwlevels')>-1) or (theURL.lower().find('/nwis/uv')>-1) or (theURL.lower().find('/nwis/peak')>-1 ) or (theURL.lower().find('/nwis/measurements')>-1)"]
#ndbc_rdb
queries_lvl1 += ["theURL.lower().find('www.ndbc.noaa.gov/view_text_file.php')>-1 "]

# url_mods = []
# url_mods += ["theURL.split('?')[0] + '?service=wfs&request=getCapabilities'"]
# url_mods += ["theURL.split('?')[0] + '?service=wms&request=getCapabilities'"]
# url_mods += ["theURL"]
# url_mods += ["theURL"]
# url_mods += ["theURL"]
# url_mods += ["theURL"]
# url_mods += ["theURL"]

queries_lvl2 = []
queries_lvl2 += ["testurl(theURL.split('?')[0] + '?service=wfs&request=getCapabilities')"]
queries_lvl2 += ["testurl(theURL.split('?')[0] + '?service=wms&request=getCapabilities')"]
queries_lvl2 += ["testurl(theURL)"]*5

queries_lvl3 = ["True"] * 4
queries_lvl3 += ["(requests.get(theURL).headers['Content-Type'].find('html')>-1)"]
queries_lvl3 += ["True"] * 2

tokens = ['wfsclient', 'wmsclient', 'kmlclient', 'MAGIC:tsv:multipart', 'webbrowser', 'nwis_rdb', 'ndbc_txt']
#tbl_dct = {'lvl1': queries_lvl1, 'url_mods': url_mods, 'lvl2': queries_lvl2, 'lvl3': queries_lvl3, 'token': tokens}
tbl_dct = {'lvl1': queries_lvl1, 'lvl2': queries_lvl2, 'lvl3': queries_lvl3, 'token': tokens}
token_assignment_df = pd.DataFrame(tbl_dct)
#token_assignment_df

In [15]:
token_assignment_df

,lvl1,lvl2,lvl3,token
0,theprotocol.lower().find('wfs')>-1 or theURL.l...,testurl(theURL.split('?')[0] + '?service=wfs&r...,True,wfsclient
1,theprotocol.lower().find('wms')>-1 or (theURL....,testurl(theURL.split('?')[0] + '?service=wms&r...,True,wmsclient
2,thedescription.lower().find('kml download')>-1...,testurl(theURL),True,kmlclient
3,(theappprofile.lower().find('pymag')>-1) or (t...,testurl(theURL),True,MAGIC:tsv:multipart
4,(theURL.lower().find('.html')>-1 ) or (theURL....,testurl(theURL),(requests.get(theURL).headers['Content-Type']....,webbrowser
5,(theURL.lower().find('/nwis/qwdata')>-1) or (t...,testurl(theURL),True,nwis_rdb
6,theURL.lower().find('www.ndbc.noaa.gov/view_te...,testurl(theURL),True,ndbc_txt


In [16]:
token_logic = {}
token_logic['wfsclient'] = ["theprotocol.lower().find('wfs')>-1 or theURL.lower().find('service=wfs')>-1",
                           "testurl(theURL.split('?')[0] + '?service=wfs&request=getCapabilities')",
                           ]
token_logic['wmsclient'] = ["theprotocol.lower().find('wms')>-1 or (theURL.lower().find('service=wms')>-1 and theURL.lower().find('request=kml')==-1)",
                           "testurl(theURL.split('?')[0] + '?service=wfs&request=getCapabilities')"]
token_logic['kmlclient'] = ["thedescription.lower().find('kml download')>-1 or (theURL.lower().find('request=kml')>-1 and theURL.lower().find('mode=download')>-1) or theURL.lower().find('.kml')>-1 or theURL.lower().find('.kmz')>-1",
                           "testurl(theURL)"]
token_logic['MAGIC:tsv:multipart'] = ["(theappprofile.lower().find('pymag')>-1) or (theappprofile.lower().find('magic:tsv:multipart')>-1) or (thename.lower().find('magic multipart tab separated values')>-1 )",
                               "testurl(theURL)"]
token_logic['webbrowser'] = ["(theURL.lower().find('.html')>-1 ) or (theURL.lower().find('.pdf')>-1) or (theURL.lower().find('.htm')>-1 ) or (theprotocol.lower().find('http')>-1)",
                            "testurl(theURL)",
                            "(requests.get(theURL).headers['Content-Type'].find('html')>-1)"]
token_logic['nwis_rdb'] = ["(theURL.lower().find('/nwis/qwdata')>-1) or (theURL.lower().find('/nwis/gwlevels')>-1) or (theURL.lower().find('/nwis/uv')>-1) or (theURL.lower().find('/nwis/peak')>-1 ) or (theURL.lower().find('/nwis/measurements')>-1)",
                          "testurl(theURL)",
                          "True"]
token_logic['ndbc_rdb'] = ["theURL.lower().find('www.ndbc.noaa.gov/view_text_file.php')>-1",
                          "testurl(theURL)",
                          "True"]

In [17]:
def decompose_logic(logic_list):
    row = {}
    row['lvl1'] = logic_list[0]
    row['lvl2'] = logic_list[1]
    try:
        row['lvl3'] = logic_list[2]
    except IndexError:
        row['lvl3'] = "True"
    return pd.Series(row)

In [18]:
token_logic_df = (pd.DataFrame(pd.Series(token_logic), columns = ['logic'])['logic']
                  .apply(lambda x: decompose_logic(x))
                  .reset_index()
                  .rename(columns = {'index': 'token'}))
token_logic_df

,token,lvl1,lvl2,lvl3
0,wfsclient,theprotocol.lower().find('wfs')>-1 or theURL.l...,testurl(theURL.split('?')[0] + '?service=wfs&r...,True
1,wmsclient,theprotocol.lower().find('wms')>-1 or (theURL....,testurl(theURL.split('?')[0] + '?service=wfs&r...,True
2,kmlclient,thedescription.lower().find('kml download')>-1...,testurl(theURL),True
3,MAGIC:tsv:multipart,(theappprofile.lower().find('pymag')>-1) or (t...,testurl(theURL),True
4,webbrowser,(theURL.lower().find('.html')>-1 ) or (theURL....,testurl(theURL),(requests.get(theURL).headers['Content-Type']....
5,nwis_rdb,(theURL.lower().find('/nwis/qwdata')>-1) or (t...,testurl(theURL),True
6,ndbc_rdb,theURL.lower().find('www.ndbc.noaa.gov/view_te...,testurl(theURL),True


Getting Table 2

In [19]:
token_url_sheet = 'https://docs.google.com/spreadsheet/ccc?key=1jFGjp2_QRT1z2YR4DJZwA-OTaj41zpdkUiX_JxBoISU&gid=123788411&output=csv'
token_url_sheet = pd.read_csv(token_url_sheet).replace(np.nan, '')
token_url_sheet = token_url_sheet.loc[token_url_sheet['token'] != '']
token_url_sheet['Label'] = [str(i) for i in range(token_url_sheet.shape[0])]

In [20]:
token_url_sheet

,token,applications,Label,Application Description,software landing page,download installer URL,Web application URL,url parameters
0,MAGIC:tsv:multipart,Jupyter notebook,0,demo notebook using the PmagPy modules from MagIC,,,still in testing,documentID: the MagIC contribution ID (5 digit...
1,MAGIC:tsv:multipart,PmagPy,1,PmagPy is a set of python modules that impleme...,https://earthref.org/PmagPy/cookbook/,,,
3,OWL-rdf/xml,protege,2,,https://github.com/protegeproject/protege-dist...,,,
4,OWL-rdf/xml,TopBraid Composer,3,"General ontologe editor. Note, will not open f...",,,,
5,NWIS-rdb-uv,Jupyter notebook,4,,,,http://suave-jupyterhub.com/user/zeppelin-v/no...,dataURL: a url that will get the rdb formatted...
6,NWIS-rdb-gwlevels,Jupyter notebook,5,,,,http://suave-jupyterhub.com/user/zeppelin-v/no...,dataURL: a url that will get the rdb formatted...
7,NWIS-rdb-peak,Jupyter notebook,6,,,,http://suave-jupyterhub.com/user/zeppelin-v/no...,dataURL: a url that will get the rdb formatted...
8,NWIS-rdb-inventory,Jupyter notebook,7,,,,http://suave-jupyterhub.com/user/zeppelin-v/no...,dataURL: a url that will get the rdb formatted...
9,NWIS-rdb-measurements,Jupyter notebook,8,,,,http://suave-jupyterhub.com/user/zeppelin-v/no...,dataURL: a url that will get the rdb formatted...
10,wfsclient,Jupyter notebook,9,"inspect service content--feature types, and pr...",,,http://suave-jupyterhub.com/user/zeppelin-v/no...,endpointURL: the base URL for the WFS service;...


Getting the token-label table

In [21]:
def create_desc(row):
    return ('label: ' + row['label']) + '\n' + ('description: ' + row['notes'])
desc_sheet_url = 'https://docs.google.com/spreadsheet/ccc?key=1jFGjp2_QRT1z2YR4DJZwA-OTaj41zpdkUiX_JxBoISU&gid=64925624&output=csv'
desc_df = pd.read_csv(desc_sheet_url)
desc_df['label'] = desc_df['label'].fillna('no label available')
desc_df['notes'] = desc_df['notes'].fillna('no description available')
desc_df['description'] = desc_df.apply(lambda x: create_desc(x), axis = 1)
#desc_df

In [22]:
token_desc_all = pd.Series(data = list(desc_df['description'].values), index = list(desc_df['token']))

### Taking in an input

Replace the value of the json_dump_str with the actual output in the notebook

In [27]:
#assume that I have a json string - one similar to that in ShutongCinergiDispatch
#this str is the result directly from Steve's ShutongCinergiDispatch notebook
#json_dump_str = '[{"adistobj": {"name": "Service Description", "url": "http://web2.nbmg.unr.edu/ArcGIS/services/CO_Data/COActiveFaults/MapServer/WMSServer?request=GetCapabilities&service=WMS", "description": "parameters:{layers:\\"ActiveFault\\"}", "protocol": "OGC:WMS", "appprofile": "", "functioncode": "381", "functiontext": "webService", "distorg": "Colorado Geological Survey", "formatlist": []}}, {"adistobj": {"name": "WFS Capabilities", "url": "http://web2.nbmg.unr.edu/ArcGIS/services/CO_Data/COActiveFaults/MapServer/WFSServer?request=GetCapabilities&service=WFS", "description": "parameters:{typeName:\\"ActiveFault\\"}", "protocol": "OGC:WFS", "appprofile": "", "functioncode": "381", "functiontext": "webService", "distorg": "Colorado Geological Survey", "formatlist": []}}, {"adistobj": {"name": "ESRI Service Endpoint", "url": "http://web2.nbmg.unr.edu/ArcGIS/rest/services/CO_Data/COActiveFaults/MapServer", "description": "", "protocol": "ESRI", "appprofile": "", "functioncode": "381", "functiontext": "webService", "distorg": "Colorado Geological Survey", "formatlist": []}}, {"adistobj": {"name": "Zipped Excel Workbook containing Active Faults Data for the State of Colorado", "url": "http://repository.stategeothermaldata.org/metadata/record/9e15e1a59b768b330d029e86dc1a10a1/file/activefaults_20131011repo.zip", "description": "", "protocol": "", "appprofile": "", "functioncode": "375", "functiontext": "download", "distorg": "Colorado Geological Survey", "formatlist": []}}, {"adistobj": {"name": "NGDS RSS feed for services notifications", "url": "http://notifications.usgin.org/", "description": "", "protocol": "", "appprofile": "", "functioncode": "375", "functiontext": "download", "distorg": "Colorado Geological Survey", "formatlist": []}}]'
#load it into json
json_list = json.loads(json_dump_str)
#transform it into a table (for visualization and as the input for dist_to_tokens)
info_list = [dist_obj['adistobj'] for dist_obj in json_list]
info_df = pd.DataFrame(info_list)
#info_df

In [28]:
# def dists_to_tokens(json_df, mapping_df):
#     #type cast the series of sets of tokens into a list of sets of tokens
#     result = json_df.apply(lambda x: features_to_tokens(x, mapping_df), axis = 1)
#     dct_lst = list(result)
#     #return the union of all the sets, since every token in each set is qualified
#     return dct_lst
# def features_to_tokens(row, mapping_df):
#     import requests
#     def testurl(theurl):
#         #try HEAD first in case the response document is big
#         #print('trying ' + theurl)
#         try:
#             r = requests.head(theurl)
#             if (r.status_code != requests.codes.ok):
#                 #check GET in case is an incomplete http implementation
#                 r = requests.get(theurl)
#                 if (r.status_code == requests.codes.ok):
#                     return True
#                 else:
#                     return False
#             else:
#                 return True
#         except:
#             return False
    
#     #getting the variables ready
#     #display(row)
#     theURL = row['url']
#     thename = row['name']
#     thedescription = row['description']
#     theprotocol = row['protocol']
#     theappprofile = row['appprofile']
#     thefunctioncode = row['functioncode']
#     thefunctiontext = row['functiontext']
#     #set the scope for the evaluation
#     test_locals = locals()
    
    
    
#     #gate1 indicates rows that passes the first level of query
#     gate1 = mapping_df['lvl1'].apply(lambda x: eval(x, test_locals))
#     display(gate1)
#     #gate2 indicates rows that passes the second level of query
#     gate2 = mapping_df['lvl2'].apply(lambda x: eval(x, test_locals))
#     display(gate2)
#     #gate3 indicates rows that passes the third level of the query
#     gate3 = mapping_df['lvl3'].apply(lambda x: eval(x, test_locals))
#     display(gate3)
    
#     #the eventual valid path to the token are the intersection of valid rows of the three gates
#     #display(gate1 & gate2 & gate3)
#     final_condition = gate1 & gate2 & gate3
    
#     #group the chosen tokens into a set
#     token_candidates = mapping_df['token'].loc[final_condition]
#     token_url_dict = {}

    
#     for token in token_candidates:
#             token_url_dict[token] = {'dist_url': theURL, 'name': thename}
#     return token_url_dict

In [29]:
def dists_to_tokens_v2(json_df, mapping_df):
    #type cast the series of sets of tokens into a list of sets of tokens
    result = json_df.apply(lambda x: features_to_tokens_v2(x, mapping_df), axis = 1)
    dct_lst = list(result)
    #return the union of all the sets, since every token in each set is qualified
    return dct_lst

def features_to_tokens_v2(row, mapping_df):
    import requests
    def testurl(theurl):
        #try HEAD first in case the response document is big
        #print('trying ' + theurl)
        try:
            r = requests.head(theurl)
            if (r.status_code != requests.codes.ok):
                #check GET in case is an incomplete http implementation
                r = requests.get(theurl)
                if (r.status_code == requests.codes.ok):
                    return True
                else:
                    return False
            else:
                return True
        except:
            return False
    
    #getting the variables ready
    #display(row)
    theURL = row['url']
    thename = row['name']
    thedescription = row['description']
    theprotocol = row['protocol']
    theappprofile = row['appprofile']
    thefunctioncode = row['functioncode']
    thefunctiontext = row['functiontext']
    #set the scope for the evaluation
    test_locals = locals()
    
    
    
    #gate1 indicates rows that passes the first level of query
    gate1 = mapping_df['lvl1'].apply(lambda x: eval(x, test_locals))
    #print('gate1:')
    #display(gate1)
    #gate2 indicates rows that passes the second level of query
    gate2 = mapping_df.loc[gate1.loc[gate1].index]['lvl2'].apply(lambda x: eval(x, test_locals))
    #print('gate2')
    #display(gate2)
    #gate3 indicates rows that passes the third level of the query
    gate3 = mapping_df.loc[gate2.loc[gate2].index]['lvl3'].apply(lambda x: eval(x, test_locals))
    #print('gate3')
    #display(gate3)
    
    #the eventual valid path to the token are the intersection of valid rows of the three gates
    #display(gate1 & gate2 & gate3)
    #final_condition = gate1 & gate2 & gate3
    
    #group the chosen tokens into a set
    try:
        token_candidates = mapping_df['token'].loc[list(gate3.loc[gate3].index)]
        token_url_dict = {}
        for token in token_candidates:
            token_url_dict[token] = {'dist_url': theURL, 'name': thename}
        return token_url_dict
    except KeyError:
        print('encounters error')
        return {}

In [30]:
def combine_dcts(dct_list):
    #combining a list of dicts into a single dict, duplicated keys will have their values in a list
    combined = {}
    for dct in dct_list:
        for key in dct.keys():
            if key in combined.keys():
                combined[key] += [dct[key]]
            else:
                combined[key] = [dct[key]]
    return combined

In [31]:
#Running the assignment function - anticlimatic one-liner
mapping_results = dists_to_tokens_v2(info_df, token_assignment_df)
#mapping_results

In [32]:
mapping_results

[{'wmsclient': {'dist_url': 'http://web2.nbmg.unr.edu/ArcGIS/services/CO_Data/COActiveFaults/MapServer/WMSServer?request=GetCapabilities&service=WMS',
   'name': 'Service Description'}},
 {'wfsclient': {'dist_url': 'http://web2.nbmg.unr.edu/ArcGIS/services/CO_Data/COActiveFaults/MapServer/WFSServer?request=GetCapabilities&service=WFS',
   'name': 'WFS Capabilities'}},
 {},
 {},
 {}]

In [33]:
#combined result
token_dist = combine_dcts(mapping_results)

In [34]:
token_dist

{'wfsclient': [{'dist_url': 'http://web2.nbmg.unr.edu/ArcGIS/services/CO_Data/COActiveFaults/MapServer/WFSServer?request=GetCapabilities&service=WFS',
   'name': 'WFS Capabilities'}],
 'wmsclient': [{'dist_url': 'http://web2.nbmg.unr.edu/ArcGIS/services/CO_Data/COActiveFaults/MapServer/WMSServer?request=GetCapabilities&service=WMS',
   'name': 'Service Description'}]}

In [35]:
def token_url_template(token, url_template, dist_url):
    final_url = re.sub('{.*}', dist_url, url_template)
    return final_url

In [36]:
def token_to_actionable_menu(token_url_sheet, token_dist):
    #return a dictionary with keys of application label and values of application's actionable url
    #the dict will be 

    #ah shit, it's gonna be ugly
    label_actionable = {}
    for token in token_dist.keys():
        #actionables is the sub-table of the token_url_sheet that only has rows regarding the specific token
        actionables = token_url_sheet.loc[token_url_sheet['token'] == token]
        dist_list = token_dist[token]
        
        #TO-DO
        for i in range(actionables.shape[0]):
            row = actionables.iloc[i, :]
            app_name = row['Label']
            url_template = row['Web application URL']
            for dist in dist_list:
                dist_name = dist['name'][:50]
                dist_url = dist['dist_url']
                menu_label = dist_name + "-" + app_name
                url = token_url_template(token, url_template, dist_url)
                label_actionable[menu_label] = url
    return label_actionable

In [37]:
menu = token_to_actionable_menu(token_url_sheet, token_dist)
menu

{'Service Description-12': '',
 'Service Description-13': '',
 'Service Description-14': '',
 'WFS Capabilities-9': 'http://suave-jupyterhub.com/user/zeppelin-v/notebooks/DispatchTesting/WFSprocessor.ipynb?endpoint=http://web2.nbmg.unr.edu/ArcGIS/services/CO_Data/COActiveFaults/MapServer/WFSServer?request=GetCapabilities&service=WFS'}

## 3. Select an operation to run##

### Presenting the menu for token and transfer url selection

In [38]:
selection = widgets.Dropdown(options = menu)
selection

Dropdown(options={'Service Description-12': '', 'Service Description-13': '', 'Service Description-14': '', 'W…

## 4. Open the URL of the selected notebook ##

In [40]:
redirect_url = selection.value
webbrowser.open_new(redirect_url)
print("If browser fails to redirect...\nnotebook link (click to open): " + '\n' + redirect_url)

If browser fails to redirect...
notebook link (click to open): 
http://suave-jupyterhub.com/user/zeppelin-v/notebooks/DispatchTesting/WFSprocessor.ipynb?endpoint=http://web2.nbmg.unr.edu/ArcGIS/services/CO_Data/COActiveFaults/MapServer/WFSServer?request=GetCapabilities&service=WFS


--------------------------------------------------

# Some extra functionality (still working on) (no idea where to plug this)

In [ ]:
#possibly record the reason for not passing a gate
#need to decide where to insert this extra information
#TO-DO
def determine_results(result_df):
    if result_df.apply(lambda x: x.all(), axis = 1).any():
        return 'success'
    elif result_df.apply(lambda x: x['lvl1'] and x['lvl2'] and not x['lvl3'], axis = 1).any():
        return 'not a valid web-page (failed to evoke web browser option)'
    elif result_df.apply(lambda x: x['lvl1'] and not x['lvl2'], axis = 1).any():
        return 'server request error (testURL failed)'
    else:
        return 'failure (did not make it pass initial screening)'

In [ ]:
result_df = pd.DataFrame(
    {'lvl1': [False, True, True, True], 'lvl2': [True, True, True, False], 'lvl3': [True, True, False, True]})
result_df